# 3. Read CSV Data and perform Basic Data Analysis #
Examples based on Chapter 2, "Spark: Definitive Guide: Big Data processing Made Simple"

In this example, sample flight data for 2010 to 2015 is processed and Simple **MAX**, **Top-N** and **Group-By** operations are performed against the data using Spark SQL.

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.\
        builder.\
        appName("pyspark-nb-3-analysis").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        config("spark.eventLog.enabled", "true").\
        config("spark.eventLog.dir", "file:///opt/workspace/events").\
        getOrCreate()     

22/01/08 16:50:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
# Read flight data in from CSV
flightData = spark.read.option("inferSchema", True).option("header", True).csv("/opt/workspace/datain/flight-data/*.csv")

#### Spark SQL Views ####
A temporary table-view can be constructed on a Spark data-frame with the `createOrReplaceTempView()` method.  

This allows the data-frame data to be analysed using ANSI SQL via the temporary table construct, which is an in-memory non-persistant view of the data built on the Spark data frame.

In [3]:
flightData.createOrReplaceTempView("flight_data")

#### Group-By Example ####
Query the data-frame and count the number of flights grouped by destination country.  
Using Spark SQL to query the temporary table `flight_data` creates another Spark data-frame (in this case, "`results`")

In [4]:
results = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data
GROUP BY DEST_COUNTRY_NAME
""")

In [5]:
# View the execution plan for running this query.  
results.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#10, 200)
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[partial_count(1)])
      +- *(1) FileScan csv [DEST_COUNTRY_NAME#10] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/opt/workspace/datain/flight-data/2010-summary.csv, file:/opt/workspace/da..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


#### Max-Value Example ####

In [6]:
# Get max flight destination from flights using Spark SQL
spark.sql("SELECT max(count) from flight_data").take(1)

[Row(max(count)=370002)]

In [7]:
# Pyspark / Python equiv
from pyspark.sql.functions import max
flightData.select(max("count")).take(1)

[Row(max(count)=370002)]

#### Top-N Example ####
##### SQL Syntax #####

In [8]:
# Find top 5 destinations
top_5_dest = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")
top_5_dest.show()

[Stage 7:==============================================>        (170 + 2) / 200]

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|          2348280|
|           Canada|            49052|
|           Mexico|            38075|
|   United Kingdom|            10946|
|            Japan|             9205|
+-----------------+-----------------+



##### Spark SQL Dataframe API Syntax #####

In [9]:
# Find top 5 destinations - DataFrame syntax
from pyspark.sql.functions import desc
flightData.groupBy("DEST_COUNTRY_NAME").sum("count")\
    .withColumnRenamed("sum(count)", "destination_total").sort(desc("destination_total"))\
    .limit(5).show()

[Stage 10:===============================================>      (176 + 3) / 200]

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|          2348280|
|           Canada|            49052|
|           Mexico|            38075|
|   United Kingdom|            10946|
|            Japan|             9205|
+-----------------+-----------------+



#### Show Execution Path ####
Aggregation happens in two parts in the execution plan - in the `partial_sum` and `sum` calls. This is because summing a list of numbers is *commutative* and Spark can perform the sum partition-by-partition.

*Commutative*: "condition that a group of quantities connected by operators gives the same result whatever the order of the quantities involved, e.g. a × b = b × a."


##### SQL Execution Plan #####

In [10]:
spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""").explain()

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[aggOrder#74L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#10,destination_total#72L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[sum(cast(count#12 as bigint))])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#10, 200)
      +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[partial_sum(cast(count#12 as bigint))])
         +- *(1) FileScan csv [DEST_COUNTRY_NAME#10,count#12] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/opt/workspace/datain/flight-data/2010-summary.csv, file:/opt/workspace/da..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>


##### Spark SQL DataFrame API Execution Plan #####

In [11]:
flightData.groupBy("DEST_COUNTRY_NAME").sum("count")\
    .withColumnRenamed("sum(count)", "destination_total").sort(desc("destination_total"))\
    .limit(5).explain()

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#87L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#10,destination_total#87L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[sum(cast(count#12 as bigint))])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#10, 200)
      +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[partial_sum(cast(count#12 as bigint))])
         +- *(1) FileScan csv [DEST_COUNTRY_NAME#10,count#12] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/opt/workspace/datain/flight-data/2010-summary.csv, file:/opt/workspace/da..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>



+ *Exactly the same execution path is used*, whether performing the operation via Spark SQL or on the data-frame API using pyspark.  

In [12]:
spark.stop()

## Part 2 - Windowing Analytic Functions ## 

In [13]:
from pyspark.sql import SparkSession
spark = SparkSession.\
        builder.\
        appName("pyspark-nb-3-analysis-pt2").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        config("spark.eventLog.enabled", "true").\
        config("spark.eventLog.dir", "file:///opt/workspace/events").\
        getOrCreate() 

In [14]:
# load data - data_download notebook downloads data locally 
retail_df = spark.read.option("inferSchema", True).option("header", True).csv("/opt/workspace/datain/retail-data/by-day/*.csv")

In [15]:
retail_df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



#### Partition-Window Rank Example
##### Find Most Popular Stock Code for each Country #####

In [16]:
retail_df.createOrReplaceTempView("retail_view")

Only show results for countries with more than 100 items to remove low-scores with lots of duplicates
##### SQL Approach #####

In [17]:
spark.sql("""
SELECT Country,StockCode, Description, count
FROM (
SELECT
    Country,
    StockCode,
    Description,
    sum(Quantity) AS count,
    rank() OVER (PARTITION BY Country ORDER BY sum(Quantity) DESC) as rank
  FROM retail_view
  GROUP BY Country, StockCode, Description
  ) WHERE RANK = 1 AND count > 100
ORDER BY Country
""").show(100,  truncate = False)

+---------------+---------+-----------------------------------+-----+
|Country        |StockCode|Description                        |count|
+---------------+---------+-----------------------------------+-----+
|Australia      |22492    |MINI PAINT SET VINTAGE             |2916 |
|Austria        |21918    |SET 12 KIDS COLOUR  CHALK STICKS   |288  |
|Belgium        |21212    |PACK OF 72 RETROSPOT CAKE CASES    |480  |
|Canada         |37370    |RETRO COFFEE MUGS ASSORTED         |504  |
|Channel Islands|21785    |RAIN PONCHO                        |407  |
|Cyprus         |22335    |HEART DECORATION PAINTED ZINC      |384  |
|Denmark        |21915    |RED  HARMONICA IN BOX              |288  |
|EIRE           |21212    |PACK OF 72 RETROSPOT CAKE CASES    |1728 |
|Finland        |84997D   |CHILDRENS CUTLERY POLKADOT PINK    |480  |
|France         |23084    |RABBIT NIGHT LIGHT                 |4023 |
|Germany        |22326    |ROUND SNACK BOXES SET OF4 WOODLAND |1218 |
|Hong Kong      |223

##### DataFrame API Approach #####
Partition data with a Window function

In [18]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Get the Country, Stock-Code, Description, Quantity-Count
# 1. pre-processing aggregation
country_sum_df = retail_df.groupBy("Country","StockCode", "Description").sum("Quantity").withColumnRenamed("sum(Quantity)", "count")

In [19]:
from pyspark.sql.functions import col
# get the most popular product for each country
# 2. Define a Window
window = Window.partitionBy("Country").orderBy(col("count").desc())
# 3. Rank over a window
country_sum_df.withColumn("row",row_number().over(window)) \
  .filter(col("row") == 1).drop("row") \
  .filter(col("count") >= 100) \
  .orderBy(col("Country")) \
  .show(100)

[Stage 9:=====================================================> (194 + 3) / 200]

+---------------+---------+--------------------+-----+
|        Country|StockCode|         Description|count|
+---------------+---------+--------------------+-----+
|      Australia|    22492|MINI PAINT SET VI...| 2916|
|        Austria|    21918|SET 12 KIDS COLOU...|  288|
|        Belgium|    21212|PACK OF 72 RETROS...|  480|
|         Canada|    37370|RETRO COFFEE MUGS...|  504|
|Channel Islands|    21785|        RAIN PONCHO |  407|
|         Cyprus|    22335|HEART DECORATION ...|  384|
|        Denmark|    21915|RED  HARMONICA IN...|  288|
|           EIRE|    21212|PACK OF 72 RETROS...| 1728|
|        Finland|   84997D|CHILDRENS CUTLERY...|  480|
|         France|    23084|  RABBIT NIGHT LIGHT| 4023|
|        Germany|    22326|ROUND SNACK BOXES...| 1218|
|      Hong Kong|    22326|ROUND SNACK BOXES...|  150|
|        Iceland|    23076|ICE CREAM SUNDAE ...|  240|
|         Israel|    20719|WOODLAND CHARLOTT...|  130|
|          Italy|   51014A|FEATHER PEN,HOT PINK|  240|
|         

In [20]:
spark.stop()